# Advent of Code 2023: coolcomputery

In [88]:
import numpy as np, math, itertools as it

In [2]:
def LINES():
    with open('in.txt') as f:
        return f.read().split('\n')

## Day 8

### 8a

In [63]:
lines=LINES()
instr=lines[0]
ways={}
for l in lines[2:]:
    v,data=l.split(' = ')
    vl,vr=data[1:-1].split(', ')
    ways[v]=(vl,vr)

n=0
v='AAA'
while v!='ZZZ':
    d=instr[n%len(instr)]
    v=ways[v][0 if d=='L' else 1]
    n+=1
n

19783

### 8b

In [106]:
lines=LINES()
instr=lines[0]
ways={}
for l in lines[2:]:
    v,data=l.split(' = ')
    vl,vr=data[1:-1].split(', ')
    ways[v]=(vl,vr)

loops=[]
for v0 in ways:
    if v0[-1]=='A':
        start=(v0,0)
        seen={start:0}
        s=start
        iback=None
        while True:
            v,i=s
            d=instr[i]
            s=(ways[v][0 if d=='L' else 1],(i+1)%len(instr))
            if s in seen:
                iback=seen[s]
                break
            seen[s]=len(seen)
        seen_list={i:s for s,i in seen.items()}
        loops.append((
            seen_list,
            iback
        ))
'''
loop=(L,b)
locations: L[0],L[1],...,L[len(L)-1],L[b],L[b+1],...
    0<=i<len(L): at location L[i] at step i
    larger i: at location L[b + (i-len(L))%(len(L)-b)]
'''
# display(loops)

def loc_at_step(L,b,i):
    n=len(L)
    return L[i] if i<n else L[b + (i-n)%(n-b)]

for i in range(max(len(L) for L,b in loops)):
    # print([loc_at_step(L,b,i) for L,b in loops])
    if all(loc_at_step(L,b,i)[0][-1]=='Z' for L,b in loops):
        print(i)
        break

'''
find some i>len(L_k) for all (L_k,b_k) in loops
s.t. all L_k[b_k + (i-len(L_k))%(len(L_k)-b_k)] end with letter Z
'''
Z_idxs=[
    [idx for idx in range(len(L)) if L[idx][0][-1]=='Z']
    for L,b in loops
]
Z_idxs

def gcd(a,b):
    return gcd(b,a) if a>b else b if a==0 else gcd(a,b%a)
def congruence_pair(r1,m1, r2,m2):
    if m1<m2:
        return congruence_pair(r2,m2, r1,m1)
    r1%=m1
    r2%=m2
    M=(m1*m2)//gcd(m1,m2)
    # find k1 s.t. m1*k1+r1 ~ r2 mod m2
    for k1 in range(M//m1):
        i=m1*k1+r1
        if i%m2 == r2:
            return i,M
    return None

sol_i,sol_M=0,1
for zidx_list in it.product(*Z_idxs):
    print(zidx_list)
    for zidx,(L,b) in zip(zidx_list,loops):
        n=len(L)
        # need (i-n)%(n-b) + b == zidx
        # ==> i ~ (zidx-b)+n mod n-b
        # ==> i = (n-b)*m + (zidx-b) for some m
        print(n,b, f'need i%{n-b}={zidx-b+n}')
        sol_i,sol_M=congruence_pair(sol_i,sol_M, zidx-b+n,n-b)
sol_i,sol_M

(11653, 19783, 19241, 16531, 12737, 14363)
11657 4 need i%11653=23306
19785 2 need i%19783=39566
19245 4 need i%19241=38482
16535 4 need i%16531=33062
12743 6 need i%12737=25474
14365 2 need i%14363=28726


(0, 9177460370549)

## Day 7

### 7a

In [46]:
card2val={
    c:i+1
    for i,c in enumerate('123456789TJQKA')
}
hands,bids=[],{}
for l in LINES():
    h,b=l.split(' ')
    h=tuple(card2val[c] for c in h)
    hands.append(h)
    bids[h]=int(b)
assert len(set(hands))==len(hands)

def ctype(h):
    mset={}
    for c in h:
        mset[c]=mset.get(c,0)+1
    freqs=sorted(mset.values())
    if freqs[-1]==3:
        return 3.5 if len(freqs)>1 and freqs[-2]==2 else 3
    if freqs[-1]==2:
        return 2.5 if len(freqs)>1 and freqs[-2]==2 else 2
    return freqs[-1]

A=sorted(sorted(hands),key=ctype)
sum(bids[h]*(i+1) for i,h in enumerate(A))#,A

249483956

### 7b

In [55]:
card2val={
    c:i
    for i,c in enumerate('J123456789T.QKA')
}
hands,bids=[],{}
for l in LINES():
    h,b=l.split(' ')
    h=tuple(card2val[c] for c in h)
    hands.append(h)
    bids[h]=int(b)
assert len(set(hands))==len(hands)

def ctype(h):
    mset={}
    for c in h:
        if c!=0:
            mset[c]=mset.get(c,0)+1
    freqs=sorted(mset.values())
    if len(freqs)==0:
        freqs=[len(h)]
    else:
        freqs[-1]+=sum(1 for c in h if c==0)
#     print(h,freqs)
    if freqs[-1]==3:
        return 3.5 if len(freqs)>1 and freqs[-2]==2 else 3
    if freqs[-1]==2:
        return 2.5 if len(freqs)>1 and freqs[-2]==2 else 2
    return freqs[-1]

A=sorted(sorted(hands),key=ctype)
sum(bids[h]*(i+1) for i,h in enumerate(A))#,[(h,ctype(h)) for h in A]

252137472

## Day 6

### 6a

In [11]:
Ts,Ds=[
    [int(tok) for tok in l.split(':')[1].split()]
    for l in LINES()
]
out=1
for t,d in zip(Ts,Ds):
    nways=sum(1 for bt in range(t+1) if (t-bt)*bt>d)
    out*=nways
out

2612736

### 6b

In [17]:
T,D=[
    int(''.join(l.split(':')[1].split()))
    for l in LINES()
]
T,D

'''
(T-x)*x > D
x**2 - Tx < -D
x**2 - Tx + D < 0
(T +/- sqrt(T**2 - 4D)) / 2
'''
hi,lo=[(T+e*(T**2-4*D)**0.5)/2 for e in [1,-1]]
int(hi)-math.ceil(lo)+1

29891250

## Day 5

### 5a

In [19]:
sections='\n'.join(LINES()).split('\n\n')
seeds=[int(tok) for tok in sections[0].split(': ')[1].split(' ')]

def conversion(s):
    rng_strs=s.split(':\n')[1].split('\n')
#     out={}
#     for tok in rng_strs:
#         dest,src,le=(int(v) for v in tok.split())
#         for i in range(le):
#             assert src+i not in out
#             out[src+i]=dest+i
#     return out
    return [tuple(int(v) for v in tok.split()) for tok in rng_strs]
def convert(rngs,v):
    for dest,src,le in rngs:
        if src<=v and v<src+le:
            return dest+(v-src)
    return v

Ds=[conversion(sec) for sec in sections[1:]]
vals=seeds
for D in Ds:
    vals=[convert(D,v) for v in vals]
min(vals)

51580674

### 5b

In [34]:
sections='\n'.join(LINES()).split('\n\n')
seed_data=[int(tok) for tok in sections[0].split(': ')[1].split(' ')]

def conversion(s):
    rng_strs=s.split(':\n')[1].split('\n')
    out=[]
    for tok in rng_strs:
        dest,src,le=(int(v) for v in tok.split())
        out.append((src,src+le,dest-src))
    return out

Ds=[conversion(sec) for sec in sections[1:]]
seed_rngs=[(src,src+le) for src,le in zip(seed_data[::2],seed_data[1::2])]

# if x in range (dest,src,le): x -> x+(dest-src)
#  src <= x < src+le

def convert_range(start,converters):
#     print(start,converters)
    # assume that converters operate over disjoint ranges
    converters=sorted(converters,key=lambda rng:rng[1])
    _clo,_chi=converters[0][0],converters[-1][1]
    _lo,_hi=min(start[0],_clo),max(start[1],_chi)
    converters=[(_lo,_clo,0)]+converters+[(_chi,_hi,0)]
    id_rngs=[
        (h0,l1,0)
        for (l0,h0,sh0),(l1,h1,sh1) in zip(converters,converters[1:])
    ]
    converters=sorted(converters+id_rngs,key=lambda rng:rng[1])
#     print(converters)
    out=[]
    for clo,chi,sh in converters:
        alo,ahi=max(start[0],clo),min(start[1],chi)
#         print(f'\t{[alo,ahi]}')
        if alo<ahi:
            out.append(
                (alo+sh,ahi+sh)
            )
    return out

rngs=seed_rngs
for D in Ds:
    rngs=[nrng for rng in rngs for nrng in convert_range(rng,D)]
#     print(rngs)
min(lo for lo,hi in rngs)#,rngs

99751240

## Day 4

### 4a

In [14]:
cards=[
    tuple(
        [int(v) for v in tok.split()]
        for tok in l.split(': ')[1].split(' | ')
    )
    for l in LINES()
]
# display(cards)
out=0
for wins,yours in cards:
    cnt=sum(1 for v in yours if v in wins)
    out+=2**(cnt - 1) if cnt>0 else 0
out

17782

### 4b

In [17]:
cards=[
    tuple(
        [int(v) for v in tok.split()]
        for tok in l.split(': ')[1].split(' | ')
    )
    for l in LINES()
]
N=len(cards)
mults=[1]*N
for i,(wins,yours) in enumerate(cards):
    cnt=sum(1 for v in yours if v in wins)
    for k in range(cnt):
        mults[i+k+1]+=mults[i]
sum(mults)

8477787

## Day 3

### 3a

In [10]:
grid=LINES()
# print(grid)
R,C=len(grid),len(grid[0])
adj=[[False]*C for r in range(R)]
dirs=[
    (1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)
]
for r in range(R):
    for c in range(C):
        s=grid[r][c]
        if s not in '.1234567890':
            for dr,dc in dirs:
                nr,nc=r+dr,c+dc
                if 0<=nr and nr<R and 0<=nc and nc<C:
                    adj[nr][nc]=True

good_nums=[]
for r in range(R):
    j=0
    while j<C:
        if grid[r][j] in '1234567890':
            k=j
            while k<C and grid[r][k] in '1234567890':
                k+=1
#             print(j,k,grid[r][j:k])
            if any(adj[r][i] for i in range(j,k)):
                good_nums.append(int(grid[r][j:k]))
            j=k
        else:
            j+=1
# print(good_nums)
sum(good_nums)

537832

### 3b

In [18]:
grid=LINES()
# print(grid)
R,C=len(grid),len(grid[0])
num_idx=[[None]*C for r in range(R)]
num_vals={}
for r in range(R):
    j=0
    while j<C:
        if grid[r][j] in '1234567890':
            k=j
            while k<C and grid[r][k] in '1234567890':
                k+=1
            n_id=len(num_vals)
            n_v=int(grid[r][j:k])
            num_vals[n_id]=n_v
            for i in range(j,k):
                num_idx[r][i]=n_id
            j=k
        else:
            j+=1

# display(num_idx)

dirs=[
    (1,0),(1,1),(0,1),(-1,1),(-1,0),(-1,-1),(0,-1),(1,-1)
]
rats=[]
for r in range(R):
    for c in range(C):
        s=grid[r][c]
        if s=='*':
            n_ids=set()
            for dr,dc in dirs:
                nr,nc=r+dr,c+dc
                if 0<=nr and nr<R and 0<=nc and nc<C:
                    n_id=num_idx[nr][nc]
                    if n_id is not None:
                        n_ids.add(n_id)
            if len(n_ids)==2:
                rats.append(math.prod(num_vals[n_id] for n_id in n_ids))
sum(rats)#,rats

81939900

## Day 2

### 2a

In [59]:
games=[s[s.find(' '):] for s in LINES()]
allow={'red':12, 'green':13, 'blue':14}
out=0
for G in games:
    gi,G=G.split(': ')
    gi=int(gi)
    G=G.split('; ')
#     print(gi,G)
    ssets=[]
    for s in G:
        D={}
        for tok in s.split(', '):
            cnt,col=tok.split(' ')
            D[col]=int(cnt)
        ssets.append(D)
    bnd={}
    for D in ssets:
        for col,v in D.items():
            bnd[col]=max(bnd.get(col,0),v)
    if all(bnd[c]<=allow[c] for c in allow):
#         print(bnd)
        out+=gi
out

2632

### 2b

In [60]:
games=[s[s.find(' '):] for s in LINES()]
allow={'red':12, 'green':13, 'blue':14}
out=0
for G in games:
    gi,G=G.split(': ')
    gi=int(gi)
    G=G.split('; ')
#     print(gi,G)
    ssets=[]
    for s in G:
        D={}
        for tok in s.split(', '):
            cnt,col=tok.split(' ')
            D[col]=int(cnt)
        ssets.append(D)
    bnd={}
    for D in ssets:
        for col,v in D.items():
            bnd[col]=max(bnd.get(col,0),v)
#     if all(bnd[c]<=allow[c] for c in allow):
#     print(bnd)
    out+=math.prod(bnd.values())
out

69629

## Day 1

In [26]:
with open('in.txt') as f:
    lines=f.read().split('\n')

### 1a

In [27]:
out=0
for l in lines:
    digs=[int(c) for c in l if c in '1234567890']
    out+=digs[0]*10+digs[-1]
out

55621

### 1b

In [28]:
D={s:i for i,s in enumerate('zero one two three four five six seven eight nine'.split())}
out=0
for l in lines:
    digs=[]
    for i in range(len(l)):
        if l[i] in '1234567890':
            digs.append(int(l[i]))
        else:
            for s,v in D.items():
                if l[i:].startswith(s):
                    digs.append(v)
    x=digs[0]*10+digs[-1]
#     print(l,x)
    out+=x
out

53592